In [1]:
%cd ~/user_data/research/git/zero-shot-object-detection/frontend_owlvit/owlv2_coco_base
%pwd

/mnt/researchfiles/ECE IMAPLE/cluster_data/user_data/jw3897/research/git/zero-shot-object-detection/frontend_owlvit/owlv2_coco_base


'/mnt/researchfiles/ECE IMAPLE/cluster_data/user_data/jw3897/research/git/zero-shot-object-detection/frontend_owlvit/owlv2_coco_base'

In [2]:
import os
import random
import numpy as np
import torch
from torch.utils.data import DataLoader
import torchvision
from torchvision.datasets import CocoDetection
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
%matplotlib inline
from PIL import Image
import cv2
from pycocotools.coco import COCO
from transformers import pipeline
from transformers import Owlv2ImageProcessor, Owlv2Processor, Owlv2ForObjectDetection
from torchmetrics.detection.mean_ap import MeanAveragePrecision
from tqdm import tqdm
import pandas as pd
import glob

random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

/home/DREXEL/jw3897/miniconda3/envs/owlvit/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
val_image_dir = r'/mnt/active_storage/Joe/coco_set/val2017/images'


novel_annotation_path = r'/mnt/active_storage/Joe/coco_set/val2017/annotations/ovd_ins_val2017_t.json'
novel_coco_val_dataset = CocoDetection(
    root=val_image_dir, 
    annFile=novel_annotation_path,
)

base_annotation_path = r'/mnt/active_storage/Joe/coco_set/val2017/annotations/ovd_ins_val2017_b.json'
base_coco_val_dataset = CocoDetection(
    root=val_image_dir, 
    annFile=base_annotation_path,
)

loading annotations into memory...
Done (t=0.04s)
creating index...
index created!
loading annotations into memory...
Done (t=0.41s)
creating index...
index created!


In [4]:
base_image_ids = []
for item in base_coco_val_dataset:
    base_image_ids.append(item[1][0]['image_id'])

novel_image_ids = []
for item in novel_coco_val_dataset:
    novel_image_ids.append(item[1][0]['image_id'])

shared_image_ids = list(set(base_image_ids).intersection(set(novel_image_ids)))
shared_image_ids.sort()

In [5]:
shared_coco_val_dataset = []
for item in novel_coco_val_dataset:
    metadata_idx = 1
    image_id = item[metadata_idx][0]['image_id']
    if image_id in shared_image_ids:
        shared_coco_val_dataset.append(item)

In [6]:
predictions_df_total = pd.read_parquet(r'./owlv2_coco_base_predictions.parquet')

In [ ]:
objectness_threshold = 0.2
score_threshold = 0.1
min_dim = 16

predictions_likely_objects = predictions_df_total[predictions_df_total['objectness_score'] >= objectness_threshold].sort_values('objectness_score', ascending=False).groupby('image_id').head(100).sort_values('image_id')
predictions_likely_objects = predictions_likely_objects.loc[predictions_likely_objects['image_id'].isin(shared_image_ids)]

known_objects = predictions_likely_objects[predictions_likely_objects['score'] >= score_threshold]
unknown_objects = predictions_likely_objects[predictions_likely_objects['score'] <= score_threshold]

display(known_objects)
display(unknown_objects)

print(f"Number of Known Annotations: {len(known_objects)}. Number of Unknown Annotations: {len(unknown_objects)}")

,image_id,label,score,objectness_score,xmin,ymin,xmax,ymax
56126,1503,mouse,0.925781,0.832371,123.469925,179.454468,157.958923,199.421616
55798,1503,mouse,0.195024,0.232760,307.235962,153.737595,319.720154,164.040314
55428,1503,toaster,0.145705,0.544008,192.091370,100.404221,292.725677,147.229950
55932,1503,laptop,0.744366,0.535144,-0.195122,98.783356,127.959633,238.997711
55165,1503,tv,0.271813,0.306206,124.326973,10.537472,239.455719,159.610916
...,...,...,...,...,...,...,...,...
156337,581100,giraffe,0.764967,0.553758,375.402985,174.903046,457.012482,379.688171
156135,581100,sheep,0.110374,0.456134,93.664841,228.587341,205.222046,305.217651
156326,581100,giraffe,0.697064,0.507842,233.851593,144.705566,388.054535,398.232666
167124,581357,person,0.306396,0.241641,255.467850,97.437080,479.816925,346.343689


,image_id,label,score,objectness_score,xmin,ymin,xmax,ymax
55279,1503,remote,0.055567,0.369445,96.540100,103.998489,120.847557,161.302979
55594,1503,mouse,0.050527,0.245349,134.416107,111.131279,205.046005,159.928574
55938,1503,mouse,0.009197,0.269581,0.308208,130.165726,317.193298,240.838318
55196,1503,horse,0.089887,0.404980,279.463531,93.532410,320.128601,152.998123
54411,1503,clock,0.010299,0.263020,268.243256,6.110020,295.819580,101.931030
...,...,...,...,...,...,...,...,...
167589,581357,laptop,0.057195,0.234230,25.962440,337.826660,107.538383,359.686493
168248,581357,bicycle,0.005295,0.646139,49.977936,327.986267,123.944717,518.138794
166413,581357,backpack,0.006384,0.302958,316.926239,114.625443,362.680420,152.093796
167888,581357,bicycle,0.015661,0.206565,61.052029,328.466461,102.281570,410.364929


Number of Known Annotations: 12395. Number of Unknown Annotations: 13878


In [8]:
final_df = pd.DataFrame(columns=[
    'image_id', 
    'known_object_labels', 
    'known_object_bboxes',
    'unknown_object_bbox',
    'unknown_object_potential_label',
    'unknown_object_class_score',
    'unknown_object_objectness'
    ])

#merged_df = pd.merge(predictions_likely_objects, known_objects)

In [9]:
dfs = []

#unknown_object = unknown_objects.iloc[0]
for idx, unknown_object in unknown_objects.iterrows():
    known_objects_in_image = known_objects[known_objects['image_id'] == unknown_object['image_id']]
    unknown_object_dict = {
        'image_id': unknown_object['image_id'],
    }

    known_object_labels = []
    known_object_bboxes = []
    for idx, known_object in known_objects_in_image.iterrows():
        known_object_labels.append(known_object['label'])
        known_object_bboxes.append([known_object['xmin'], known_object['ymin'], known_object['xmax'], known_object['ymax']])

    unknown_object_bbox = [unknown_object['xmin'], unknown_object['ymin'], unknown_object['xmax'], unknown_object['ymax']]

    unknown_object_dict['known_object_labels'] = known_object_labels
    unknown_object_dict['known_object_bboxes'] = known_object_bboxes
    unknown_object_dict['unknown_object_bbox'] = unknown_object_bbox
    unknown_object_dict['unknown_object_potential_label'] = unknown_object['label']
    unknown_object_dict['unknown_class_score'] = unknown_object['score']
    unknown_object_dict['unknown_object_objectness'] = unknown_object['objectness_score']


    df = pd.DataFrame(columns=[
    'image_id', 
    'known_object_labels', 
    'known_object_bboxes',
    'unknown_object_bbox',
    'unknown_object_potential_label',
    'unknown_object_class_score',
    'unknown_object_objectness'
    ])

    df.loc[0] = [
        unknown_object_dict['image_id'], 
        unknown_object_dict['known_object_labels'], 
        unknown_object_dict['known_object_bboxes'], 
        unknown_object_dict['unknown_object_bbox'], 
        unknown_object_dict['unknown_object_potential_label'],
        unknown_object_dict['unknown_class_score'],
        unknown_object_dict['unknown_object_objectness']
        ]

    dfs.append(df)


final_df = pd.concat(dfs)

In [10]:
final_df['above_mindim'] = ((final_df['unknown_object_bbox'].str[2] - final_df['unknown_object_bbox'].str[0]) > min_dim)  & ((final_df['unknown_object_bbox'].str[3] - final_df['unknown_object_bbox'].str[1]) > min_dim)

In [11]:
print(f"Objectness Threshold: {objectness_threshold}; Score Threshold: {score_threshold}")
display(final_df)

Objectness Threshold: 0.2; Score Threshold: 0.1


,image_id,known_object_labels,known_object_bboxes,unknown_object_bbox,unknown_object_potential_label,unknown_object_class_score,unknown_object_objectness,above_mindim
0,1503,"[mouse, mouse, toaster, laptop, tv, tv, remote]","[[123.46992492675781, 179.4544677734375, 157.9...","[96.54010009765625, 103.99848937988281, 120.84...",remote,0.055567,0.369445,True
0,1503,"[mouse, mouse, toaster, laptop, tv, tv, remote]","[[123.46992492675781, 179.4544677734375, 157.9...","[134.41610717773438, 111.13127899169922, 205.0...",mouse,0.050527,0.245349,True
0,1503,"[mouse, mouse, toaster, laptop, tv, tv, remote]","[[123.46992492675781, 179.4544677734375, 157.9...","[0.3082084655761719, 130.1657257080078, 317.19...",mouse,0.009197,0.269581,True
0,1503,"[mouse, mouse, toaster, laptop, tv, tv, remote]","[[123.46992492675781, 179.4544677734375, 157.9...","[279.4635314941406, 93.53240966796875, 320.128...",horse,0.089887,0.404980,True
0,1503,"[mouse, mouse, toaster, laptop, tv, tv, remote]","[[123.46992492675781, 179.4544677734375, 157.9...","[268.2432556152344, 6.110019683837891, 295.819...",clock,0.010299,0.263020,True
...,...,...,...,...,...,...,...,...
0,581357,"[person, backpack]","[[255.4678497314453, 97.43708038330078, 479.81...","[25.962440490722656, 337.82666015625, 107.5383...",laptop,0.057195,0.234230,True
0,581357,"[person, backpack]","[[255.4678497314453, 97.43708038330078, 479.81...","[49.977935791015625, 327.98626708984375, 123.9...",bicycle,0.005295,0.646139,True
0,581357,"[person, backpack]","[[255.4678497314453, 97.43708038330078, 479.81...","[316.9262390136719, 114.62544250488281, 362.68...",backpack,0.006384,0.302958,True
0,581357,"[person, backpack]","[[255.4678497314453, 97.43708038330078, 479.81...","[61.05202865600586, 328.4664611816406, 102.281...",bicycle,0.015661,0.206565,True


In [12]:
final_df.to_parquet(r'/mnt/archive/owlvit_results/cocoshared_preparedforadam_obj2cls1_withmindim16.parquet')